In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "2"
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import Input,Conv2D, MaxPooling2D, UpSampling2D, Dropout, Cropping2D, concatenate, Activation,Conv2DTranspose
from tensorflow.keras.layers import BatchNormalization,add,Add,multiply,Lambda
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, History, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import backend as K
from keras_unet_collection import models, base, utils,losses
# from tensorflow.keras.utils.multi_gpu_utils import multi_gpu_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np
import os
import glob
import shutil
import cv2
from scipy import ndimage
from sklearn.model_selection import train_test_split,KFold
import matplotlib.pyplot as plt
from numba import jit
# config = tf.ConfigProto()
# # config.gpu_options.per_process_gpu_memory_fraction=0.25
# config.gpu_options.allow_growth=True
# K.set_session(tf.Session(config=config))

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2023-04-25 15:04:35.182489: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-25 15:04:35.732859: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22325 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:86:00.0, compute capability: 8.6


In [3]:
def load_data(npy_path):
    print('-'*30)
    print('load images...')
    print('-'*30)
    
    for i, path in enumerate(npy_path):
#         print(path)
        mask_npy_path = path.replace('_HE','_mask')
        train_npy_path = path
    
        imgs_tmp = [cv2.imread(train_npy_path)]
        imgs_mask_tmp = [cv2.imread(mask_npy_path,0)]
#         print(imgs_tmp.shape, imgs_mask_tmp.shape)
        
        if i==0:
            imgs = imgs_tmp
            imgs_mask = imgs_mask_tmp
            
        else:
            imgs = np.append(imgs, imgs_tmp,axis=0)
            imgs_mask = np.append(imgs_mask, imgs_mask_tmp,axis=0)
    imgs_tmp,imgs_mask_tmp = 0,0
    print('-'*30)
    print('imgs : {} \nmasks : {}'.format(imgs.shape, imgs_mask.shape))    
    print('-'*30)
    imgs = imgs.astype('float32')
    imgs_mask = imgs_mask.astype('float32')
    print('img : ', imgs.max())
    print('mask : ',imgs_mask.max())

    print('-'*30)
    print('normalization start...')
    print('-'*30)
    
    imgs = cv2.normalize(imgs, None, 0, 1, cv2.NORM_MINMAX)

    imgs_mask[imgs_mask<= 127] = 0
    imgs_mask[imgs_mask > 127] = 1

    print('img : ',imgs.max())
    print('mask : ',imgs_mask.max())

    return imgs, imgs_mask

In [4]:
def mkfolder(folder):
    if not os.path.lexists(folder):
        os.makedirs(folder)
@jit
def AttnBlock2D(x, g, inter_channel, data_format='channels_last'):

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(x)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(g)

    f = Activation('relu')(add([theta_x, phi_g]))

    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format=data_format)(f)

    rate = Activation('sigmoid')(psi_f)

    att_x = multiply([x, rate])

    return att_x

def attention_up_and_concate(down_layer, layer, data_format='channels_last'):
    
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]
    
    up = UpSampling2D(size=(2,2), data_format=data_format)(down_layer)
    layer = AttnBlock2D(x=layer, g=up, inter_channel=in_channel // 4, data_format=data_format)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: K.concatenate([x[0], x[1]], axis=3))
    
    concate = my_concat([up, layer])
    return concate
@jit
def get_att_unet(img_rows, img_cols):
    inputs = Input((img_rows, img_cols,3))
    conv1 = Conv2D(32, (3, 3), activation=None, padding='same')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    conv1 = Conv2D(32, (3, 3), activation=None, padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Activation('relu')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(64, (3, 3), activation=None, padding='same')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    conv2 = Conv2D(64, (3, 3), activation=None, padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Activation('relu')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, (3, 3), activation=None, padding='same')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    conv3 = Conv2D(128, (3, 3), activation=None, padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Activation('relu')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, (3, 3), activation=None, padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    conv4 = Conv2D(256, (3, 3), activation=None, padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Activation('relu')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Conv2D(512, (3, 3), activation=None, padding='same')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)
    conv5 = Conv2D(512, (3, 3), activation=None, padding='same')(conv5)
    conv5 = BatchNormalization()(conv5)
    conv5 = Activation('relu')(conv5)

    up6 = attention_up_and_concate(conv5, conv4)
    conv6 = Conv2D(256, (3, 3), activation=None, padding='same')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)
    conv6 = Conv2D(256, (3, 3), activation=None, padding='same')(conv6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Activation('relu')(conv6)

    up7 = attention_up_and_concate(conv6, conv3)
    conv7 = Conv2D(128, (3, 3), activation=None, padding='same')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)
    conv7 = Conv2D(128, (3, 3), activation=None, padding='same')(conv7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Activation('relu')(conv7)

    up8 = attention_up_and_concate(conv7, conv2)
    conv8 = Conv2D(64, (3, 3), activation=None, padding='same')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)
    conv8 = Conv2D(64, (3, 3), activation=None, padding='same')(conv8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Activation('relu')(conv8)

    up9 = attention_up_and_concate(conv8, conv1)
    conv9 = Conv2D(32, (3, 3), activation=None, padding='same')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)
    conv9 = Conv2D(32, (3, 3), activation=None, padding='same')(conv9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Activation('relu')(conv9)

    conv10 = Conv2D(1, (1, 1), activation='sigmoid')(conv9)

    model = Model(inputs=inputs, outputs=conv10)

    return model

In [5]:
def dice_coef(y_true, y_pred, smooth=1e-6):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

def recall(y_true, y_pred):
    # clip(t, clip_value_min, clip_value_max) : clip_value_min~clip_value_max 이외 가장자리를 깎아 낸다
    # round : 반올림한다
    y_true_yn = K.round(K.clip(y_true, 0, 1)) # 실제값을 0(Negative) 또는 1(Positive)로 설정한다
    y_pred_yn = K.round(K.clip(y_pred, 0, 1)) # 예측값을 0(Negative) 또는 1(Positive)로 설정한다

    # True Positive는 실제 값과 예측 값이 모두 1(Positive)인 경우이다
    count_true_positive = K.sum(y_true_yn * y_pred_yn) 

    # (True Positive + False Negative) = 실제 값이 1(Positive) 전체
    count_true_positive_false_negative = K.sum(y_true_yn)

    # Recall =  (True Positive) / (True Positive + False Negative)
    # K.epsilon()는 'divide by zero error' 예방차원에서 작은 수를 더한다
    recall = count_true_positive / (count_true_positive_false_negative + K.epsilon())

    # return a single tensor value
    return recall

In [6]:
img_path = 'ROI/New/'
all_patient = np.array([i.replace('_HE.png', '') for i in os.listdir(img_path) if '_HE.png' in i])

In [7]:
all_patient

array(['248', '357', '89', ..., '540', '541', '542'], dtype='<U10')

In [8]:
# train_patient = ['2120737','2091197','2181633','2219487','2013713','2202545','2169164','2228384','2204600','2167546','2097782','2213245','2205867','2195929','2108941','2197912','2232197','2091069','0867365','2172317']
# val_patient = ['2098098','2132226','2022898','2090880','2213752']
# test_patient = ['0790305','2200290','2103280','2192712','2193810','2075521','2206976']
# add2_patient = ['2120737','2091197','2181633','2219487','2013713','2202545','2169164','2228384','2204600','2167546','2097782','2213245','2205867','2195929','2108941','2197912','2232197','2091069','0867365','2172317','2193810','2075521','2206976']
kf = KFold(n_splits=5, random_state=5, shuffle=True)
fold = []
for train_index, test_index in kf.split(all_patient):
    train_patient, test_patient = all_patient[train_index.astype(int)], all_patient[test_index.astype(int)]    
    
    testset = [img_path+'{}_HE.png'.format(p) for p in test_patient]
    trainset = [img_path+'{}_HE.png'.format(p) for p in train_patient]

    fold.append([trainset, testset])

In [9]:
fold

[[['ROI/New/248_HE.png',
   'ROI/New/357_HE.png',
   'ROI/New/89_HE.png',
   'ROI/New/1013_HE.png',
   'ROI/New/269_HE.png',
   'ROI/New/620_HE.png',
   'ROI/New/1075_HE.png',
   'ROI/New/1043_HE.png',
   'ROI/New/191_HE.png',
   'ROI/New/1125_HE.png',
   'ROI/New/1161_HE.png',
   'ROI/New/845_HE.png',
   'ROI/New/622_HE.png',
   'ROI/New/650_HE.png',
   'ROI/New/228_HE.png',
   'ROI/New/225_HE.png',
   'ROI/New/246_HE.png',
   'ROI/New/242_HE.png',
   'ROI/New/259_HE.png',
   'ROI/New/261_HE.png',
   'ROI/New/268_HE.png',
   'ROI/New/275_HE.png',
   'ROI/New/266_HE.png',
   'ROI/New/288_HE.png',
   'ROI/New/297_HE.png',
   'ROI/New/353_HE.png',
   'ROI/New/369_HE.png',
   'ROI/New/389_HE.png',
   'ROI/New/398_HE.png',
   'ROI/New/35_HE.png',
   'ROI/New/31_HE.png',
   'ROI/New/67_HE.png',
   'ROI/New/61_HE.png',
   'ROI/New/80_HE.png',
   'ROI/New/143_HE.png',
   'ROI/New/149_HE.png',
   'ROI/New/179_HE.png',
   'ROI/New/165_HE.png',
   'ROI/New/902_HE.png',
   'ROI/New/1580000_HE.png

In [10]:
# # train_patient = ['2120737','2091197','2181633','2219487','2013713','2202545','2169164','2228384','2204600','2167546','2097782','2213245','2205867','2195929','2108941','2197912','2232197','2091069','0867365','2172317']
# # val_patient = ['2098098','2132226','2022898','2090880','2213752']
# # test_patient = ['0790305','2200290','2103280','2192712','2193810','2075521','2206976']

# # train_patient,test_patient = train_test_split(all_patient,test_size=0.2)
# kf = KFold(n_splits=5, random_state=5, shuffle=True)
# fold = []
# for train_index, test_index in kf.split(all_patient):
#     train_patient, test_patient = all_patient[train_index.astype(int)], all_patient[test_index.astype(int)]
#     train_patient, val_patient = train_test_split(train_patient,test_size=0.2)
#     testset = [npy_path+'{}_mask.npy'.format(p) for p in test_patient]
#     print(test_patient, train_patient)
#     trainset = [npy_path+'{}_mask.npy'.format(p) for p in train_patient]
#     valset = [npy_path+'{}_mask.npy'.format(p) for p in val_patient]
#     fold.append({'trainset':trainset, 'valset':valset, 'testset':testset})
#     print(trainset)

In [11]:
data_gen_args = dict(rotation_range=10.,
                    width_shift_range=0.1,
                    height_shift_range=0.1,
                    zoom_range=0.2, horizontal_flip=True)

In [12]:
for foldnum in range(0,5):
    trainset = fold[foldnum][0]
    testset = fold[foldnum][1]
    
    sv_model_folder ='6_result/exp_fold{}/model/'.format(foldnum)
    mkfolder(sv_model_folder)

    sv_pred_folder = '6_result/exp_fold{}/pred/'.format(foldnum)
    mkfolder(sv_pred_folder)

    imgs_train, imgs_mask_train = load_data(trainset)
    print('='*30)
    # imgs_test, imgs_mask_test = load_data(testset[:4])
    # imgs_add, imgs_mask_add = load_data(testset[4:])
    print('-'*30)
    print("load unet model")
    print('-'*30)

    imgs_mask_train = np.expand_dims(imgs_mask_train,axis=-1)
    imgs_mask_train.shape

    imgs_train,imgs_val,imgs_mask_train,imgs_mask_val = train_test_split(imgs_train,imgs_mask_train,test_size=0.2,random_state=7)

#     image_datagen = ImageDataGenerator(**data_gen_args)
#     mask_datagen = ImageDataGenerator(**data_gen_args)

#     val_image_datagen = ImageDataGenerator()
#     val_mask_datagen = ImageDataGenerator()

#     image_generator = image_datagen.flow(imgs_train,batch_size=1,seed=1)
#     mask_generator = mask_datagen.flow(imgs_mask_train,batch_size=1,seed=1)

#     valt_generator = val_image_datagen.flow(imgs_val,batch_size=1,seed=1)
#     valm_generator = val_mask_datagen.flow(imgs_mask_val,batch_size=1,seed=1)

#     train_generator = zip(image_generator, mask_generator)
#     validation_generator = zip(valt_generator,valm_generator)
    
    
    img_rows, img_cols=512,512

    model = get_att_unet(img_rows, img_cols)
#     model = models.swin_unet_2d((512, 512, 3), filter_num_begin=64, n_labels=1, depth=4, stack_num_down=2, stack_num_up=2, 
#                             patch_size=(2, 2), num_heads=[4, 8, 8, 8], window_size=[4, 2, 2, 2], num_mlp=512, 
#                             output_activation='Softmax', shift_window=True, name='swin_unet')

#     model = models.att_unet_2d((img_rows, img_cols,3), filter_num=[64, 128, 256, 512], n_labels=1, 
#                                stack_num_down=2, stack_num_up=2, activation='ReLU', 
#                                atten_activation='ReLU', attention='add', output_activation='Sigmoid', 
#                                batch_norm=True, pool=False, unpool=False, 
#                                backbone='ResNet101V2', weights='imagenet', 
#                                freeze_backbone=True, freeze_batch_norm=True, 
#                                name='attunet')
    #     model = multi_gpu_model(model,gpus=4)

    learning_rate = 0.0001

    # model.compile(optimizer=Adam(learning_rate=learning_rate), 
    #               loss=[losses.dice,losses.dice,losses.dice,losses.dice,losses.dice,losses.dice],
    #               loss_weights=[0.25,0.25,0.25,0.25,1,0],
    #               metrics=['acc', 'binary_crossentropy', recall])

    model.compile(optimizer=Adam(learning_rate=learning_rate), 
                  loss=[losses.dice],
                  metrics=['acc', 'binary_crossentropy', recall])

    # reference : https://stackoverflow.com/questions/43782409/how-to-use-modelcheckpoint-with-custom-metrics-in-keras
    # print(model.metrics_names)

    # sv_model_folder ='../4_result/exp3_RGBE/model/fold{}/'.format(num)
    # mkfolder(sv_model_folder)

    # sv_pred_folder = '../4_result/exp3_HE/pred/fold{}/'.format(num)
    # mkfolder(sv_pred_folder)

    save_check_folder = sv_model_folder+'hdf5/'
    mkfolder(save_check_folder)

    def sch(epoch):
        if epoch>30:
            return 0.001
        else:
            return 0.01

    epochs = 100
    batch_size = 1

    model_checkpoint = ModelCheckpoint(sv_model_folder+'best.h5', monitor='val_loss', verbose=1, save_best_only=True)
    sc = LearningRateScheduler(sch)
    reduceLROnPlateau = ReduceLROnPlateau(monitor='val_loss', patience=5, factor=0.1, verbose=1)
    earlystopping = EarlyStopping(monitor='val_loss', patience=20, verbose=1)

    print('-'*30)
    print('start training')
    print('-'*30)

    # imgs_train = np.rollaxis(imgs_train, 3, 1)
    # imgs_mask_train = np.rollaxis(imgs_mask_train, 3, 1)
    # imgs_validation = np.rollaxis(imgs_validation, 3, 1)
    # imgs_mask_validation = np.rollaxis(imgs_mask_validation, 3, 1)

#     model.fit([imgs_train,imgs_mask_train], steps_per_epoch=752,epochs=epochs, verbose=1, validation_data=[imgs_val,imgs_mask_val],
#               validation_steps = 188,shuffle=True, callbacks=[model_checkpoint,reduceLROnPlateau,earlystopping])
    
    model.fit(imgs_train, imgs_mask_train, batch_size=4, epochs=epochs, verbose=1, shuffle=True,
              validation_data=(imgs_val,imgs_mask_val), callbacks=[model_checkpoint,reduceLROnPlateau,earlystopping])


    print('save model')

    model.save(sv_model_folder+'last.h5'.format(learning_rate, epochs))

    # print('predict test data')
    # imgs_mask_test = model.predict(imgs_test, batch_size=4, verbose=1)

    # pred_file_name = sv_pred_folder +'exp1.npy'
    # np.save(pred_file_name, imgs_mask_test)
    # #     num += 1

------------------------------
load images...
------------------------------
------------------------------
imgs : (940, 512, 512, 3) 
masks : (940, 512, 512)
------------------------------
img :  253.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
------------------------------
load unet model
------------------------------


/tmp/ipykernel_3098183/3082630395.py:38: NumbaWarning: 
Compilation is falling back to object mode WITH looplifting enabled because Function "get_att_unet" failed type inference due to: Untyped global name 'Input': Cannot determine Numba type of <class 'function'>

File "../../../../../../tmp/ipykernel_3098183/3082630395.py", line 40:
<source missing, REPL/exec in use?>

  @jit
/data01/anaconda3/envs/jeon2/lib/python3.7/site-packages/numba/core/object_mode_passes.py:152: NumbaWarning: Function "get_att_unet" was compiled in object mode without forceobj=True.

File "../../../../../../tmp/ipykernel_3098183/3082630395.py", line 38:
<source missing, REPL/exec in use?>

  state.func_ir.loc))
/data01/anaconda3/envs/jeon2/lib/python3.7/site-packages/numba/core/object_mode_passes.py:162: NumbaDeprecationWarning: 
Fall-back from the nopython compilation path to the object mode compilation path has been detected, this is deprecated behaviour.

For more information visit https://numba.readthedocs

------------------------------
start training
------------------------------


2023-03-22 10:33:59.534377: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/100


...

188/188 [==============================] - 29s 152ms/step - loss: 0.0237 - acc: 0.9902 - binary_crossentropy: 0.0535 - recall: 0.9813 - val_loss: 0.0493 - val_acc: 0.9770 - val_binary_crossentropy: 0.1437 - val_recall: 0.9594

Epoch 00023: val_loss did not improve from 0.04912
Epoch 24/100
188/188 [==============================] - 28s 148ms/step - loss: 0.0234 - acc: 0.9904 - binary_crossentropy: 0.0524 - recall: 0.9815 - val_loss: 0.0490 - val_acc: 0.9771 - val_binary_crossentropy: 0.1419 - val_recall: 0.9572

Epoch 00024: val_loss improved from 0.04912 to 0.04903, saving model to 6_result/exp_fold1/model/best.h5
Epoch 25/100
188/188 [==============================] - 28s 149ms/step - loss: 0.0229 - acc: 0.9905 - binary_crossentropy: 0.0517 - recall: 0.9818 - val_loss: 0.0492 - val_acc: 0.9771 - val_binary_crossentropy: 0.1432 - val_recall: 0.9560

Epoch 00025: val_loss did not improve from 0.04903
Epoch 26/100
188/188 [==============================] - 28s 149ms/step - loss: 0.0225 

188/188 [==============================] - 28s 150ms/step - loss: 0.0392 - acc: 0.9841 - binary_crossentropy: 0.0783 - recall: 0.9715 - val_loss: 0.0484 - val_acc: 0.9787 - val_binary_crossentropy: 0.1104 - val_recall: 0.9667

Epoch 00012: val_loss improved from 0.05808 to 0.04839, saving model to 6_result/exp_fold2/model/best.h5
Epoch 13/100
188/188 [==============================] - 28s 151ms/step - loss: 0.0365 - acc: 0.9847 - binary_crossentropy: 0.0737 - recall: 0.9744 - val_loss: 0.1171 - val_acc: 0.9476 - val_binary_crossentropy: 0.2122 - val_recall: 0.8211

Epoch 00013: val_loss did not improve from 0.04839
Epoch 14/100
188/188 [==============================] - 28s 151ms/step - loss: 0.0338 - acc: 0.9855 - binary_crossentropy: 0.0737 - recall: 0.9744 - val_loss: 0.0447 - val_acc: 0.9793 - val_binary_crossentropy: 0.1075 - val_recall: 0.9589

Epoch 00014: val_loss improved from 0.04839 to 0.04471, saving model to 6_result/exp_fold2/model/best.h5
Epoch 15/100
188/188 [==========

188/188 [==============================] - 28s 149ms/step - loss: 0.0147 - acc: 0.9925 - binary_crossentropy: 0.0565 - recall: 0.9841 - val_loss: 0.0364 - val_acc: 0.9809 - val_binary_crossentropy: 0.1716 - val_recall: 0.9639

Epoch 00064: val_loss did not improve from 0.03630
Epoch 65/100
188/188 [==============================] - 28s 148ms/step - loss: 0.0148 - acc: 0.9925 - binary_crossentropy: 0.0563 - recall: 0.9839 - val_loss: 0.0364 - val_acc: 0.9809 - val_binary_crossentropy: 0.1715 - val_recall: 0.9639

Epoch 00065: val_loss did not improve from 0.03630
Epoch 66/100
188/188 [==============================] - 28s 149ms/step - loss: 0.0153 - acc: 0.9924 - binary_crossentropy: 0.0546 - recall: 0.9835 - val_loss: 0.0364 - val_acc: 0.9809 - val_binary_crossentropy: 0.1715 - val_recall: 0.9639

Epoch 00066: val_loss did not improve from 0.03630
Epoch 67/100
188/188 [==============================] - 28s 149ms/step - loss: 0.0148 - acc: 0.9925 - binary_crossentropy: 0.0566 - recall: 

188/188 [==============================] - 29s 153ms/step - loss: 0.0267 - acc: 0.9874 - binary_crossentropy: 0.0718 - recall: 0.9772 - val_loss: 0.0408 - val_acc: 0.9802 - val_binary_crossentropy: 0.1214 - val_recall: 0.9640

Epoch 00020: val_loss improved from 0.04632 to 0.04076, saving model to 6_result/exp_fold3/model/best.h5
Epoch 21/100
188/188 [==============================] - 28s 150ms/step - loss: 0.0250 - acc: 0.9884 - binary_crossentropy: 0.0653 - recall: 0.9777 - val_loss: 0.0399 - val_acc: 0.9806 - val_binary_crossentropy: 0.1189 - val_recall: 0.9629

Epoch 00021: val_loss improved from 0.04076 to 0.03993, saving model to 6_result/exp_fold3/model/best.h5
Epoch 22/100
188/188 [==============================] - 28s 149ms/step - loss: 0.0242 - acc: 0.9887 - binary_crossentropy: 0.0629 - recall: 0.9794 - val_loss: 0.0398 - val_acc: 0.9806 - val_binary_crossentropy: 0.1207 - val_recall: 0.9645

Epoch 00022: val_loss improved from 0.03993 to 0.03981, saving model to 6_result/ex

Epoch 22/100
188/188 [==============================] - 28s 151ms/step - loss: 0.0198 - acc: 0.9905 - binary_crossentropy: 0.0496 - recall: 0.9813 - val_loss: 0.0383 - val_acc: 0.9806 - val_binary_crossentropy: 0.1145 - val_recall: 0.9629

Epoch 00022: val_loss improved from 0.03909 to 0.03834, saving model to 6_result/exp_fold4/model/best.h5
Epoch 23/100
188/188 [==============================] - 28s 149ms/step - loss: 0.0188 - acc: 0.9910 - binary_crossentropy: 0.0475 - recall: 0.9819 - val_loss: 0.0386 - val_acc: 0.9803 - val_binary_crossentropy: 0.1192 - val_recall: 0.9633

Epoch 00023: val_loss did not improve from 0.03834
Epoch 24/100
188/188 [==============================] - 28s 150ms/step - loss: 0.0180 - acc: 0.9913 - binary_crossentropy: 0.0456 - recall: 0.9824 - val_loss: 0.0385 - val_acc: 0.9803 - val_binary_crossentropy: 0.1229 - val_recall: 0.9643

Epoch 00024: val_loss did not improve from 0.03834
Epoch 25/100
188/188 [==============================] - 28s 150ms/step - 

In [13]:
imgs_val.shape

(188, 512, 512, 3)

In [14]:
# num = 3
# for dataset in fold[2:3]:
#     trainset = dataset['trainset']
#     valset = dataset['valset']
#     testset = dataset['testset']
# # load datasets




In [15]:
# imgs_test, imgs_mask_test = 0,0
image_generator = 0


In [16]:
imgs_mask_test = np.expand_dims(imgs_mask_test,axis=-1)

NameError: name 'imgs_mask_test' is not defined

In [10]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import r2_score
import cv2
from tensorflow.keras.models import load_model

In [13]:
for foldnum in range(0,5):
    
    save = []
    
    testset = fold[foldnum][1]
    imgs_test, imgs_mask_test = load_data(testset)
    model = load_model('6_result/exp_fold{}/model/best.h5'.format(foldnum), custom_objects={"dice": losses.dice, 'recall':recall})
    print(model.summary())
    # imgs_test, imgs_mask_test = load_data(testset[:4])
    mask_pred = model.predict(imgs_test, batch_size=4, verbose=1)

    print(testset[0])
    # true_list=np.load(testset[0])
    true_list = imgs_mask_test
    true_list=true_list.astype('float32')
    # true_list = true_list/255.0
    # true_list[true_list > 0.5] = 1
    # true_list[true_list <= 0.5] = 0
    print(true_list.shape)

    pred_list=mask_pred
    # pred_list=imgs_mask_test
    pred_list[pred_list > 0.5] = 1
    pred_list[pred_list <= 0.5] = 0
    print(pred_list.shape)

    # for i in range(pred_list.shape[0]):
    #     pred = pred_list[i].astype('uint8')
    #     pred[pred <= 0.5] = 0
    #     pred[pred > 0.5] = 255
    # #     pred = fill_hole_cv(pred)
    #     pred_list[i]=pred

    # pred_list[pred_list > 127] = 1
    # pred_list[pred_list <= 127] = 0

    sensitivity=[]
    specificity=[]
    acc=[]
    dsc=[]

    for i in range(len(true_list)):
        yt=true_list[i].flatten()
        yp=pred_list[i].flatten()
        mat=confusion_matrix(yt,yp)
        if len(mat) == 2:
            ac=(mat[1,1]+mat[0,0])/(mat[1,0]+mat[1,1]+mat[0,1]+mat[0,0])
            st=mat[1,1]/(mat[1,0]+mat[1,1])
            sp=mat[0,0]/(mat[0,1]+mat[0,0])
            if mat[1,0]+mat[1,1] == 0:
                specificity.append(sp)
                acc.append(ac)
            else:
                sensitivity.append(st)  
                specificity.append(sp)
                acc.append(ac)
        else:
            specificity.append(1)
            acc.append(1)

    for i in range(len(true_list)):
        yt=true_list[i]
        yp=pred_list[i]
        if np.sum(yt) != 0 and np.sum(yp) != 0:
            dice = np.sum(yp[yt==1])*2.0 / (np.sum(yt) + np.sum(yp))
            dsc.append(dice)

    print("complete")      
    print("acc avg : {0:0.4f}".format(np.mean(acc)))
    print("sensitivity avg : {0:0.4f}".format(np.mean(sensitivity)))
    print("specificity avg : {0:0.4f}".format(np.mean(specificity)))
    print("dsc avg : {0:0.4f}".format(np.mean(dsc)))
    print('-'*30)
    print("sensitivity min:",np.min(sensitivity))
    print("specificity min:",np.min(specificity))
    print("dsc min:",np.min(dsc))
    print("acc min:",np.min(acc))
    print('-'*30)
    print("sensitivity max:",np.max(sensitivity))
    print("specificity max:",np.max(specificity))
    print("dsc max:",np.max(dsc))
    print("acc max:",np.max(acc))
    
    print(np.std(sensitivity))
    print(np.std(specificity))
    print(np.std(dsc))
    print(np.std(acc))
    
    imgs_test, imgs_mask_test = 0,0

------------------------------
load images...
------------------------------
------------------------------
imgs : (235, 512, 512, 3) 
masks : (235, 512, 512)
------------------------------
img :  255.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 512, 512, 32) 896         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 512, 512, 32) 128         c

59/59 [==============================] - 2s 37ms/step
ROI/New/1059_HE.png
(235, 512, 512)
(235, 512, 512, 1)
complete
acc avg : 0.9797
sensitivity avg : 0.9574
specificity avg : 0.9872
dsc avg : 0.9646
------------------------------
sensitivity min: 0.0
specificity min: 0.8520112894489267
dsc min: 0.7732653937220476
acc min: 0.7928733825683594
------------------------------
sensitivity max: 0.9918463621671204
specificity max: 1.0
dsc max: 0.9888226072439278
acc max: 0.993743896484375
0.09190463434716721
0.014677443566516669
0.024964006336898714
0.021979072558522033
------------------------------
load images...
------------------------------
------------------------------
imgs : (235, 512, 512, 3) 
masks : (235, 512, 512)
------------------------------
img :  253.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
Model: "model_1"
______________________________________________________________________________________

59/59 [==============================] - 2s 37ms/step
ROI/New/1013_HE.png
(235, 512, 512)
(235, 512, 512, 1)
complete
acc avg : 0.9799
sensitivity avg : 0.9647
specificity avg : 0.9856
dsc avg : 0.9628
------------------------------
sensitivity min: 0.8125992985401241
specificity min: 0.933319253623147
dsc min: 0.8753487421704926
acc min: 0.9330596923828125
------------------------------
sensitivity max: 0.9923507717524929
specificity max: 0.998171311159625
dsc max: 0.9872812483068755
acc max: 0.9932670593261719
0.025239487133078355
0.012103142396694942
0.021711678440218582
0.012186228674320702
------------------------------
load images...
------------------------------
------------------------------
imgs : (235, 512, 512, 3) 
masks : (235, 512, 512)
------------------------------
img :  253.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
Model: "model_2"
________________________________________________________

59/59 [==============================] - 3s 37ms/step
ROI/New/620_HE.png
(235, 512, 512)
(235, 512, 512, 1)
complete
acc avg : 0.9801
sensitivity avg : 0.9642
specificity avg : 0.9858
dsc avg : 0.9647
------------------------------
sensitivity min: 0.0
specificity min: 0.9220816128654775
dsc min: 0.8526224585474701
acc min: 0.7767143249511719
------------------------------
sensitivity max: 0.993572708732464
specificity max: 1.0
dsc max: 0.98816167511545
acc max: 0.9939956665039062
0.06621864195032522
0.011270912653949316
0.018968425303324567
0.017243892338747544
------------------------------
load images...
------------------------------
------------------------------
imgs : (235, 512, 512, 3) 
masks : (235, 512, 512)
------------------------------
img :  252.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
Model: "model_3"
_________________________________________________________________________________________

59/59 [==============================] - 3s 38ms/step
ROI/New/248_HE.png
(235, 512, 512)
(235, 512, 512, 1)
complete
acc avg : 0.9793
sensitivity avg : 0.9582
specificity avg : 0.9868
dsc avg : 0.9630
------------------------------
sensitivity min: 0.0
specificity min: 0.9281976265040383
dsc min: 0.8808281099803776
acc min: 0.7989082336425781
------------------------------
sensitivity max: 0.995096419690831
specificity max: 1.0
dsc max: 0.9877058425445522
acc max: 0.9947013854980469
0.06836925753970499
0.011055987687031324
0.02004752686339413
0.016349057565449094
------------------------------
load images...
------------------------------
------------------------------
imgs : (235, 512, 512, 3) 
masks : (235, 512, 512)
------------------------------
img :  252.0
mask :  255.0
------------------------------
normalization start...
------------------------------
img :  1.0
mask :  1.0
Model: "model_4"
________________________________________________________________________________________

59/59 [==============================] - 2s 37ms/step
ROI/New/357_HE.png
(235, 512, 512)
(235, 512, 512, 1)
complete
acc avg : 0.9799
sensitivity avg : 0.9599
specificity avg : 0.9868
dsc avg : 0.9654
------------------------------
sensitivity min: 0.0
specificity min: 0.9397713943320825
dsc min: 0.8507699183358388
acc min: 0.7907676696777344
------------------------------
sensitivity max: 0.993123904543616
specificity max: 1.0
dsc max: 0.9888142028836779
acc max: 0.9941215515136719
0.09214175688144088
0.010103637504218572
0.020030816718856984
0.020632770120066226


In [12]:
mask_pred

array([[[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]]],


       [[[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        [[0.],
         [0.],
         [0.],
         ...,
         [0.],
         [0.],
         [0.]],

        ...,

        [[0.],
 